In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor

In [8]:
# Load Data
# load measurements.csv
measurements = pd.read_csv('../data/pop_scale/pop_scale_segmentations_measurements/measurements.csv')

# make a "genotype" column
measurements["image_name"] = [name.split('-') for name in measurements["image_name"]]
measurements.rename(columns={"image_name": "genotype"}, inplace=True)
measurements.head()

,genotype,seed_count,wing_area,env_area,seed_area,wing_perimeter,env_perimeter,seed_perimeter,wing_to_total_area,env_to_total_area,...,env_B,seed_r,seed_g,seed_b,seed_h,seed_s,seed_v,seed_l,seed_a,seed_B
0,"[pred_1.tt8t x fc6, 1.B.WBP, 2022.W_0.png]",10.0,1.093654,0.330268,0.161344,1123.685425,643.026479,1161.643794,0.689887,0.208336,...,129.462098,36.445407,33.678663,34.873153,112.331334,23.542486,36.819282,34.121071,129.593691,127.578252
1,"[pred_1.tt8t x fc6, 1.B.WBP, 2022.W_1.png]",8.0,0.964245,0.337849,0.136344,1074.271211,622.884343,919.678282,0.670342,0.234872,...,130.527530,35.791535,33.298239,33.369479,72.296793,24.167192,36.068743,33.487119,129.200841,128.339642
2,"[pred_1.tt8t x fc6, 1.B.WBP, 2022.W_2.png]",9.0,1.042220,0.320465,0.155053,1095.744299,626.256926,1033.259018,0.686693,0.211147,...,130.071972,36.896209,34.795423,35.691516,94.988904,20.732779,37.416089,35.295192,129.201687,127.665280
3,"[pred_1.tt8t x fc6, 1.B.WBP, 2022.W_3.png]",10.0,1.082148,0.363458,0.158028,1148.472222,662.683333,1086.229581,0.674810,0.226646,...,128.426014,37.857337,35.582445,37.325130,124.449762,21.882626,38.629168,36.346450,129.463484,127.156725
4,"[pred_1.tt8t x fc6, 1.B.WBP, 2022.W_4.png]",10.0,1.046019,0.336182,0.162276,1105.383910,648.825469,1132.957503,0.677264,0.217667,...,128.720987,36.846383,33.564329,35.227057,122.581005,26.961900,37.334953,34.189619,129.951850,127.382551


In [57]:
# get mean measurements for each genotype
avg_measurements = measurements.groupby("genotype").mean(numeric_only=True)
avg_measurements.reset_index(inplace=True)
avg_measurements.head()

,genotype,seed_count,wing_area,env_area,seed_area,wing_perimeter,env_perimeter,seed_perimeter,wing_to_total_area,env_to_total_area,...,env_B,seed_r,seed_g,seed_b,seed_h,seed_s,seed_v,seed_l,seed_a,seed_B
0,1.tt8t x fc6-1.B.WBP-2022.W,9.2,1.035435,0.331851,0.155475,1105.339841,639.011046,1071.990389,0.679911,0.218204,...,129.382851,37.555748,34.310118,35.473404,109.531651,26.933693,37.956240,35.037593,129.822646,127.715577
1,10.tt8t x fc5-2.B.WBP-2022.W,9.3,1.161073,0.400954,0.147018,1169.199848,683.831356,1017.650988,0.679219,0.234893,...,132.856196,37.896384,40.870943,35.259370,50.252804,37.323366,41.079685,40.921979,125.402063,131.212300
2,100.ISU397A.B.ISU Wild.S,7.7,0.856427,0.280958,0.101233,996.326933,576.985353,767.323953,0.690344,0.227520,...,136.217687,34.325431,42.763473,33.657821,59.203966,59.447930,42.763473,41.724891,122.057323,132.712178
3,101.ISU398A.B.ISU Wild.W,7.5,0.623864,0.342791,0.114140,944.005908,617.177846,841.608225,0.576592,0.318143,...,134.086315,34.021777,44.187624,36.483149,67.363022,60.369513,44.187624,43.163519,121.635036,131.507837
4,102.ISU399A.B.ISU Wild.W,8.0,0.857869,0.297062,0.119175,1009.593838,590.297842,858.097669,0.672359,0.233535,...,135.728340,40.008973,46.954682,36.738727,52.568083,56.855583,46.954682,46.983974,122.556859,133.557601


In [58]:
# load yield data
yield_data = pd.read_csv('../data/external/external_phenotypes.csv')
yield_data = yield_data.dropna(subset=['genotype'])
avg_yield_data = yield_data.groupby("genotype").mean(numeric_only=True)
avg_yield_data.reset_index(inplace=True)
avg_yield_data.head()

,genotype,plot_id,block,row,col,replicate,year,plot_rating_at_maturity,tillers,floral_branch,...,f1,f3,f2,tgw,seed_area,seed_width,seed_length,lw_ratio,Latitude,Longitude
0,15_06,224.333333,1.0,14.333333,8.000000,2.0,2023.0,1.475310,1.608673,1.680823,...,22.630408,21.958143,33.593279,0.996598,1.684563,1.364027,1.782291,1.310053,48.37998,-114.21333
1,18_01,422.333333,1.0,12.333333,14.666667,2.0,2023.0,0.908591,1.170617,1.760532,...,20.041642,23.795200,30.829575,1.015522,1.719416,1.375712,1.802536,1.313543,43.80267,-70.82433
2,19_1,455.500000,1.0,5.500000,16.000000,1.5,2023.0,0.752253,1.418739,1.310776,...,14.808205,22.463598,26.282052,1.122767,1.828578,1.415607,1.852830,1.313036,40.06340,-105.92310
3,7_10_#134,620.000000,1.0,20.000000,21.000000,1.0,2023.0,1.886705,0.891438,1.954981,...,11.270436,33.576846,26.341351,1.092068,1.810218,1.396998,1.871473,1.342429,NaN,NaN
4,950011,183.666667,1.0,13.666667,6.666667,2.0,2023.0,1.111841,0.989391,1.997681,...,18.008214,26.366161,24.143030,0.980481,1.735407,1.370303,1.824381,1.334133,44.62810,5.52312


In [ ]:
# Prepare Data: Shapes should be x: (n_samples, n_features), y: (n_samples, 1)

In [ ]:
# train decision tree

# options
max_depth = 5

# train - look up sklearn documentation for DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=max_depth)
dt.fit(x, y)

# print tree
tree = sklearn.tree.export_text(dt, feature_names=f)
print(tree)